In [ ]:
import pandas as pd

df_verif_0 = pd.read_csv("sft_eval_0.csv")
df_verif_1 = pd.read_csv("sft_eval_1.csv")
df_verif_2 = pd.read_csv("sft_eval_2.csv")
df_verif_3 = pd.read_csv("sft_eval_3.csv")
df_verif_4 = pd.read_csv("sft_eval_4.csv")

df_combined = df_verif_0.copy()
df_combined.columns = ['vuln_code', 'ground_truth', 'llm_output_0']
df_combined['llm_output_1'] = df_verif_1['llm_output']
df_combined['llm_output_2'] = df_verif_2['llm_output']
df_combined['llm_output_3'] = df_verif_3['llm_output']
df_combined['llm_output_4'] = df_verif_4['llm_output']

def extract_answer(text: str) -> str | None:
    answer = text.split("</think>\n\n")[-1]
    if "<think>" in answer:
        return None
    return answer.strip()

df_combined['llm_output_0'] = df_combined['llm_output_0'].apply(extract_answer)
df_combined['llm_output_1'] = df_combined['llm_output_1'].apply(extract_answer)
df_combined['llm_output_2'] = df_combined['llm_output_2'].apply(extract_answer)
df_combined['llm_output_3'] = df_combined['llm_output_3'].apply(extract_answer)
df_combined['llm_output_4'] = df_combined['llm_output_4'].apply(extract_answer)

In [2]:
template = """We have several vulnerability analysis for the smart contract security audit.
Your task is to produce a vulnerability explanation that selects and integrates only the most accurate, logical, reasonable, coherent and credible vulnerabilities from the existing analyses.
    
Make sure to make only one vulnerability explanation in your final answer.
Also make sure your final answer only contain the vulnerability explanation, without any additional commentary.

###Smart Contract Under Review  
```solidity
{code}
````

###Analysis 1
{output_0}

###Analysis 2
{output_1}

###Analysis 3
{output_2}

###Analysis 4
{output_3}

###Analysis 5:
{output_4}

Final Answer:
"""

df_combined["prompt"] = df_combined.apply(lambda row: template.format(
    code=row["vuln_code"],
    output_0=row.get("llm_output_0", ""),
    output_1=row.get("llm_output_1", ""),
    output_2=row.get("llm_output_2", ""),
    output_3=row.get("llm_output_3", ""),
    output_4=row.get("llm_output_4", "")
), axis=1)

In [3]:
from unsloth import FastLanguageModel, is_bfloat16_supported
import torch
max_seq_length = 32768 # Can increase for longer reasoning traces

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/Qwen3-4B",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    gpu_memory_utilization = 0.5, # Reduce if out of memory
)

/usr/local/lib/python3.10/dist-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.


/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


🦥 Unsloth Zoo will now patch everything to make training faster!


/usr/local/lib/python3.10/dist-packages/torch/utils/_pytree.py:185: FutureWarning: optree is installed but the version is too old to support PyTorch Dynamo in C++ pytree. C++ pytree support is disabled. Please consider upgrading optree using `python3 -m pip install --upgrade 'optree>=0.13.0'`.
  warnings.warn(


...


Capturing CUDA graph shapes: 100%|████████████████████████████████████████████████████████| 35/35 [00:43<00:00,  1.24s/it]

INFO 06-07 01:27:15 [model_runner.py:1592] Graph capturing finished in 43 secs, took 0.83 GiB


INFO 06-07 01:27:15 [llm_engine.py:437] init engine (profile, create kv cache, warmup model) took 55.54 seconds


Unsloth: Just some info: will skip parsing ['post_feedforward_layernorm', 'pre_feedforward_layernorm']


Unsloth: Just some info: will skip parsing ['post_feedforward_layernorm', 'pre_feedforward_layernorm']


In [4]:
messages = [
    {"role": "system", "content": "You are an expert in smart-contract security audits."},
    {"role": "user", "content": df_combined['prompt'][0]},
]

max_completion_length = 8192

text = tokenizer.apply_chat_template(
    messages,
    add_generation_prompt = True, # Must add for generation
    tokenize = False,
)

from vllm import SamplingParams
sampling_params = SamplingParams(
    temperature = 0.6,
    top_k = 20,
    top_p = 0.95,
    min_p = 0,
    presence_penalty=0.5,
    max_tokens = max_completion_length,
)
output = model.fast_generate(
    [text],
    sampling_params = sampling_params,
)[0].outputs[0].text


Processed prompts:   0%|                        | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


Processed prompts: 100%|███████████████| 1/1 [00:59<00:00, 59.23s/it, est. speed input: 9.71 toks/s, output: 58.55 toks/s]


Processed prompts: 100%|███████████████| 1/1 [00:59<00:00, 59.23s/it, est. speed input: 9.71 toks/s, output: 58.55 toks/s]

In [5]:
def extract_answer(text: str) -> str | None:
    answer = text.split("</think>\n\n")[-1]
    return answer.strip()

In [6]:
print(f"LLM Response: {extract_answer(output)}")

LLM Response: The `syncDeps` function in the Vault contract retrieves the `vusd` address from the registry and casts it to `IERC20`. If the registry returns an address that is not an ERC20 token (e.g., a non-compliant token), the cast will fail, causing the function to revert. This vulnerability arises because the code assumes the registry's `vusd()` function returns a valid ERC20 token, but there is no validation to ensure this. If the registry is compromised or misconfigured, this could lead to incorrect behavior or transaction failures.


In [7]:
df_combined['ground_truth'][0]

"The `Governance` address can call `InsuranceFund.sol#syncDeps()` to change the contract address of `vusd` anytime.\n\nHowever, since the tx to set a new address for `vusd` can get in between users' txs to deposit and withdraw, in some edge cases, it can result in users' loss of funds.\n\n### Proof of Concept\n\n1.  Alice deposited `1,000,000 VUSD` to `InsuranceFund`;\n2.  Gov called `syncDeps()` and set `vusd` to the address of `VUSDv2`;\n3.  Alice called `withdraw()` with all the `shares` and get back `0 VUSDv2`.\n\nAs a result, Alice suffered a fund loss of `1,000,000 VUSD`."

In [8]:
from tqdm import tqdm
from vllm import SamplingParams
import numpy as np

# Sampling parameters
sampling_params = SamplingParams(
    temperature=0.6,
    top_k=20,
    top_p=0.95,
    min_p=0,
    presence_penalty=0.5,
    max_tokens=8192,
)

# Create an empty list to store the outputs
final_outputs = []

# Iterate through all prompts with a progress bar
for prompt in tqdm(df_combined['prompt'], desc="Generating responses"):
    messages = [
        {"role": "system", "content": "You are an expert in smart-contract security audits."},
        {"role": "user", "content": prompt},
    ]

    # Apply chat template
    text = tokenizer.apply_chat_template(
        messages,
        add_generation_prompt=True,
        tokenize=False,
    )

    # Generate output
    output = model.fast_generate(
        [text],
        sampling_params=sampling_params,
    )[0].outputs[0].text

    # Append the result
    final_outputs.append(output)

# Assign the results to a new column
df_combined['final_output'] = final_outputs


Generating responses:   0%|                                                                       | 0/143 [00:00<?, ?it/s]


Processed prompts:   0%|                        | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


Processed prompts: 100%|██████████████| 1/1 [00:43<00:00, 43.20s/it, est. speed input: 13.31 toks/s, output: 58.91 toks/s]


Processed prompts: 100%|██████████████| 1/1 [00:43<00:00, 43.20s/it, est. speed input: 13.31 toks/s, output: 58.91 toks/s]



Generating responses:   1%|▍                                                            | 1/143 [00:43<1:42:15, 43.21s/it]

...


Processed prompts:   0%|                        | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


Processed prompts: 100%|██████████████| 1/1 [00:32<00:00, 32.12s/it, est. speed input: 38.70 toks/s, output: 58.53 toks/s]


Processed prompts: 100%|██████████████| 1/1 [00:32<00:00, 32.12s/it, est. speed input: 38.70 toks/s, output: 58.53 toks/s]



Generating responses: 100%|███████████████████████████████████████████████████████████| 143/143 [1:43:35<00:00, 38.05s/it]


Generating responses: 100%|███████████████████████████████████████████████████████████| 143/143 [1:43:35<00:00, 43.47s/it]

In [ ]:
df_combined.to_csv("vuln_combination.csv", index=False)

In [ ]:
import pandas as pd
import pickle

df_combined = pd.read_csv("vuln_combination.csv")

def extract_answer(text: str) -> str | None:
    answer = text.split("</think>\n\n")[-1]
    return answer.strip()

df_combined['clean_output'] = df_combined['final_output'].apply(extract_answer)

In [11]:
none_list = []

for i, x in enumerate(df_combined['clean_output']):
    if "<think>" in x:
        print(i)
        none_list.append(i)

92


In [12]:
def extract_answer(text: str) -> str | None:
    answer = text.split("</think>\n\n")[-1]
    return answer.strip()

In [ ]:
new_out_list = []

if len(none_list) > 0:
    for idx in none_list:
        success = False
        attempt = 0

        while not success and attempt < 3:
            try:
                messages = [
                    {"role": "system", "content": "You are an expert in smart-contract security audits."},
                    {"role": "user", "content": df_combined['prompt'][idx]},
                ]

                max_completion_length = 8192

                text = tokenizer.apply_chat_template(
                    messages,
                    add_generation_prompt=True,
                    tokenize=False,
                )

                from vllm import SamplingParams
                sampling_params = SamplingParams(
                    temperature=0.6,
                    top_k=20,
                    top_p=0.95,
                    min_p=0,
                    presence_penalty=0.5,
                    max_tokens=max_completion_length,
                )

                output = model.fast_generate(
                    [text],
                    sampling_params=sampling_params,
                )[0].outputs[0].text

                # Check if result is valid
                if "<think>" not in extract_answer(output):
                    new_out_list.append(output)
                    success = True
                else:
                    attempt += 1
                    print(f"Invalid output on attempt {attempt} for index {idx}")

            except Exception as e:
                attempt += 1
                print(f"Error on attempt {attempt} for index {idx}: {e}")

        if not success:
            print(f"Giving up on index {idx} after 3 attempts.")
            new_out_list.append(None)
            
    for idx, value in enumerate(none_list):
        df_combined['final_output'][value] = new_out_list[idx]
    
    df_combined.drop(columns=['clean_output']).to_csv("vuln_combination_v5.csv", index=False)


Processed prompts:   0%|                        | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


Processed prompts: 100%|██████████████| 1/1 [06:39<00:00, 399.98s/it, est. speed input: 6.41 toks/s, output: 20.48 toks/s]


Processed prompts: 100%|██████████████| 1/1 [06:39<00:00, 399.98s/it, est. speed input: 6.41 toks/s, output: 20.48 toks/s]

Invalid output on attempt 1 for index 92



Processed prompts:   0%|                        | 0/1 [00:00<?, ?it/s, est. speed input: 0.00 toks/s, output: 0.00 toks/s]


Processed prompts: 100%|██████████████| 1/1 [00:56<00:00, 56.50s/it, est. speed input: 45.38 toks/s, output: 56.95 toks/s]


Processed prompts: 100%|██████████████| 1/1 [00:56<00:00, 56.51s/it, est. speed input: 45.38 toks/s, output: 56.95 toks/s]